In [118]:
import json
import os
from os.path import join
from ibm_watson import PersonalityInsightsV3
import csv
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pandas as pd 
from numpy import genfromtxt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Authenticating with IBM watson service

In [73]:
# Authentication via IAM
authenticator = IAMAuthenticator('4NzkwQMWUZFocdXsxxDgNFydqc3GT0tS0LR66C1ph4Fa')
service = PersonalityInsightsV3(
    version='2018-08-01',
    authenticator=authenticator)
service.set_service_url('https://api.eu-gb.personality-insights.watson.cloud.ibm.com/instances/f44f56f2-44aa-48b1-9801-51481956814c')

## Fetching big 5 personlaity scores using the IBM watson api

In [74]:
###########################
# Profile with CSV output #
###########################
def get_song_persona(filename):
    
    with open(join(os.getcwd(), f'lyrics/{filename}'), 'r') as profile_json:
        response = service.profile(
            profile_json.read(),
            accept='text/csv',
            csv_headers=True).get_result()
    
    profile = response.content
    cr = csv.reader(profile.decode('utf-8').splitlines())
    my_list = list(cr)
    return my_list

## Calculating participant's OCEAN score using John & Shrivastava personality scoring

In [123]:
my_data = genfromtxt('big-5 data.csv', delimiter=',')
n, p = my_data.shape
ocean_matrix = np.array((n, 5))
ocean_matrix = []
ocean_matrix.append(my_data[:,5]+my_data[:,10]+my_data[:,15]+my_data[:,20]+my_data[:,25]+my_data[:,30]+(6-my_data[:,35])+my_data[:,40]+(6-my_data[:,41])+my_data[:,44])
ocean_matrix.append(my_data[:,3]+(6-my_data[:,8])+my_data[:,13]+(6-my_data[:,18])+(6-my_data[:,23])+my_data[:,28]+my_data[:,33]+my_data[:,38]+(6-my_data[:,43]))
ocean_matrix.append(my_data[:,1]+(6-my_data[:,6])+my_data[:,11]+my_data[:,16]+(6-my_data[:,21])+my_data[:,26]+(6-my_data[:,31])+my_data[:,36])
ocean_matrix.append(6-my_data[:,2]+my_data[:,7]+(6-my_data[:,12])+my_data[:,17]+my_data[:,22]+(6-my_data[:,27])+my_data[:,32]+(6-my_data[:,37])+my_data[:,42]) 
ocean_matrix.append(my_data[:,4]+(6-my_data[:,9])+my_data[:,14]+my_data[:,19]+(6-my_data[:,24])+my_data[:,29]+(6-my_data[:,34])+my_data[:,39])
ocean_matrix = (np.array(ocean_matrix)/100).T

In [124]:
## Generating a pandas dataframe consisting ocean score for each participant
ocean_df = pd.DataFrame(data=ocean_matrix, columns=['openness','conscientiousness', 'extraversion', 'agreeableness', 'neuroticism'])
demographics_df = pd.read_csv("demographics_data.csv")
demographics_df = demographics_df.drop(columns=['id','Which type of music do you prefer listening to?', 'How often do you listen to music with English lyrics?'])
final_df_1 = pd.concat([demographics_df, ocean_df], axis=1)
final_df_1.to_csv('participants_ocean.csv')
final_df_1

,age group,Sex,openness,conscientiousness,extraversion,agreeableness,neuroticism
0,24-26 years,Female,0.47,0.26,0.21,0.34,0.26
1,24-26 years,Male,0.43,0.35,0.24,0.41,0.23
2,24-26 years,Female,0.33,0.24,0.27,0.34,0.28
3,24-26 years,Male,0.36,0.38,0.30,0.41,0.18
4,27-30 years,Male,0.42,0.33,0.27,0.33,0.16
...,...,...,...,...,...,...,...
57,21-23 years,Male,0.34,0.28,0.25,0.29,0.26
58,21-23 years,Male,0.25,0.24,0.19,0.28,0.29
59,21-23 years,Male,0.40,0.38,0.34,0.42,0.14
60,21-23 years,Male,0.32,0.29,0.23,0.33,0.20


In [138]:
traits = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']
max_trait = np.argmax(ocean_matrix, axis=1)
# max_trait = [traits[i] for i in max_trait]
max_trait_df = pd.DataFrame(data=max_trait, columns=['max_trait'])
max_trait_df

,max_trait
0,0
1,0
2,3
3,3
4,0
...,...
57,0
58,4
59,3
60,3


## Fetching songs lyrics using AZLIRYCS api

In [38]:
import re
import urllib.request
from bs4 import BeautifulSoup
def get_lyrics(artist,song_title):
    # remove all except alphanumeric characters from artist and song_title
    artist = re.sub('[^A-Za-z0-9]+', "", artist)
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who
        artist = artist[3:]
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"
    
    try:
        content = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(content, 'html.parser')
        lyrics = str(soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br/>','').replace('<i>','').replace('</i>','').replace('<div>','').replace('</div>','').strip()
        return lyrics
    except Exception as e:
        return False

In [121]:
print(get_lyrics("powfu", "death bed"))

Don't stay awake for too long
Don't go to bed
I'll make a cup of coffee for your head
It'll get you up and going out of bed

Yeah...
I don't wanna fall asleep, I don't wanna pass away
I been thinking of our future cause I'll never see those days
I don't know why this has happened, but I probably deserve it
I tried to do my best, but you know that I'm not perfect
I been praying for forgiveness, you been praying for my health
When I leave this earth, hoping you'll find someone else
Cause yeah, we still young there's so much we haven't done
Getting married, start a family, watch your husband with his son
I wish it could be me, but I won't make it off this bed
I hope I go to heaven, so I see you once again
My life was kinda short, but I got so many blessings
Happy you were mine, it sucks that it's all ending

Don't stay awake for too long
Don't go to bed
I'll make a cup of coffee for your head
It'll get you up and going out of bed

And I, don't stay awake for too long
Don't go to bed
I'll 

In [ ]:
# Parsing song details from the responses filled by the participants
def get_song_details(song):
    title, artist = song.split(":")
    title = title.lower().strip()
    artist = artist.lower().strip()
    return title, artist

## Saving participant's song lyrics as txt file 

In [51]:
import csv
import os
import numpy as np

BASE_DIR = "resources"

num_lyrics = np.zeros(62)

with open('songs_data.csv') as csvDataFile:
    songs_data = csv.reader(csvDataFile)
    
    for idx, songs_row in enumerate(songs_data):
        for i in range(1, len(songs_row)):
            title, artist = get_song_details(songs_row[i])
            if(title != "na" and artist != "na"):
                filename = f'{title} by {artist}.txt'
                filename = filename.replace(' ','_')
                print(f'{idx}, {filename}')
                if not os.path.isfile(os.path.join(BASE_DIR, filename)):
                    lyrics = get_lyrics(artist, title)
                    if lyrics:
                        num_lyrics[idx] += 1
                        with open(os.path.join(BASE_DIR, filename), 'w') as f:
                            f.write(lyrics)
                else:
                    num_lyrics[idx]+=1

38, death_bed_by_powfu.txt
38, words_by_gregory_alan_isakov.txt
38, in_your_eyes_by_the_weeknd.txt
38, feel_good_inc_by_gorillaz.txt
38, yeah_right_by_joji.txt
38, faith_by_the_weeknd.txt
38, hardest_to_love_by_the_weeknd.txt
38, wish_you_were_sober_by_conan_gray.txt
38, hurt_by_johny_cash.txt
38, highway_to_hell_by_ac_dc.txt
39, lovers_on_the_sun_by_david_guetta.txt
39, starving_by_zedd.txt
39, blow_your_mind_by_dua_lipa.txt
39, one_kiss_by_dua_lipa.txt
39, without_me_by_halsey.txt
39, right_now_by_nick_jonas.txt
39, ignite_by_alan_walker.txt
39, on_my_way_by_alan_walker.txt
39, dynamite_by_taio_cruz.txt
39, lush_life_by_zarra_larson.txt
40, let_her_go_by_the_passengers.txt
40, ricky_by_denzel_curry.txt
40, sad_by_xxxtentacion.txt
40, clout_cobain_by_denzel_curry.txt
40, wonderwall_by_oasis.txt
40, it's_time_by_imagine_dragons.txt
41, believer_by_imagine_dragons.txt
41, the_nights_by_avicii.txt
41, here_comes_the_sun_by_beatles.txt
41, astronaut_by_julian_emery.txt
41, feel_good_by_da

## Contatinating lyrics in a single file for each individual

In [66]:
import csv
import os
import numpy as np

BASE_DIR = "resources"

num_lyrics = np.zeros(62)

with open('songs_data.csv') as csvDataFile:
    songs_data = csv.reader(csvDataFile)
    
    for idx, songs_row in enumerate(songs_data):
        lf = open(f'lyrics/lyrics_{idx}.txt', 'a+')
        
        for i in range(1, len(songs_row)):
            title, artist = get_song_details(songs_row[i])
            filename = f'{title} by {artist}.txt'
            filename = filename.replace(' ','_')
            lf.write("fdcs")
            if os.path.isfile(os.path.join(BASE_DIR, filename)):
                with open(os.path.join(BASE_DIR, filename), 'r') as f:
                            lf.write(f.read())
        lf.close()

## Generating song's lyrics persona using the IBM watson personality insight api

In [76]:
lyrics_persona = []
num_participants = 62
for i in range(num_participants):
    ocean_list = get_song_persona(f'lyrics_{i}.txt')
    lyrics_persona.append(ocean_list[1])
print(ocean_list)

[['big5_agreeableness', 'facet_altruism', 'facet_cooperation', 'facet_modesty', 'facet_morality', 'facet_sympathy', 'facet_trust', 'big5_conscientiousness', 'facet_achievement_striving', 'facet_cautiousness', 'facet_dutifulness', 'facet_orderliness', 'facet_self_discipline', 'facet_self_efficacy', 'big5_extraversion', 'facet_activity_level', 'facet_assertiveness', 'facet_cheerfulness', 'facet_excitement_seeking', 'facet_friendliness', 'facet_gregariousness', 'big5_neuroticism', 'facet_anger', 'facet_anxiety', 'facet_depression', 'facet_immoderation', 'facet_self_consciousness', 'facet_vulnerability', 'big5_openness', 'facet_adventurousness', 'facet_artistic_interests', 'facet_emotionality', 'facet_imagination', 'facet_intellect', 'facet_liberalism', 'need_liberty', 'need_ideal', 'need_love', 'need_practicality', 'need_self_expression', 'need_stability', 'need_structure', 'need_challenge', 'need_closeness', 'need_curiosity', 'need_excitement', 'need_harmony', 'value_conservation', 'valu

In [129]:
## Generating a pandas dataframe consisting ocean score for each participant's song's lyrics
df = pd.DataFrame(lyrics_persona, columns=ocean_list[0]) 
persona_df = df[['big5_openness', 'big5_conscientiousness', 'big5_extraversion', 'big5_agreeableness', 'big5_neuroticism']]
final_df_2 = pd.concat([demographics_df, persona_df], axis=1)
final_df_2.to_csv('lyrics_ocean.csv')
final_df_2

,age group,Sex,big5_openness,big5_conscientiousness,big5_extraversion,big5_agreeableness,big5_neuroticism
0,24-26 years,Female,0.5266808255725408,0.85121544203933,0.431858007717562,0.9164615084904904,0.9715535815763532
1,24-26 years,Male,0.2713759187826197,0.639525679652127,0.3919121843067217,0.7723047254411164,0.9451417049359991
2,24-26 years,Female,0.7769830752728288,0.7896044934588473,0.85869408831672,0.9881644906431961,0.8955405826900422
3,24-26 years,Male,0.18339135298561443,0.11964831669820775,0.5022656167865905,0.24746299495747248,0.8488159757884786
4,27-30 years,Male,0.5852090582382109,0.22693536821345828,0.12429150684211371,0.04882340705400362,0.9398648841362773
...,...,...,...,...,...,...,...
57,21-23 years,Male,0.8736189966404528,0.7841996463607356,0.5599234668193237,0.9170240728296242,0.9723481603323543
58,21-23 years,Male,0.47848353663666643,0.2583891094055292,0.019302529897321452,0.6347347770925464,0.7912422162921219
59,21-23 years,Male,0.649814824144349,0.012382092194648364,0.0549703497131358,0.4587200219896725,0.9973258648776249
60,21-23 years,Male,0.2959689735679725,0.8200312902651901,0.1640047152340296,0.8518833561056718,4.678406069508201E-5


In [139]:
model_df = pd.concat([persona_df, max_trait_df], axis=1)
model_df.to_csv('naive_bayes.csv', index=False)